In [1]:
import fitz
import json

In [ ]:
file_path = '../pdfs/7enYF2uL5kFZlOOpQhLl0nUT91RjCbeR.pdf'

In [ ]:
def parse_pdf_to_paragraphs(file_path):
    with fitz.open(file_path) as pdf_document:
        result = []
        document_name = file_path.split('/')[-1]

        for page_number in range(pdf_document.page_count):
            page = pdf_document.load_page(page_number)
            page_data = page.get_text("dict")

            current_paragraph = []
            previous_indent = None
            paragraph_number = 1

            for block in page_data['blocks']:
                if 'lines' not in block:
                    continue

                current_indent = block['bbox'][0]
                text = ''
                for line in block['lines']:
                    text += line['spans'][0]['text'].lstrip()

                if previous_indent is None:
                    previous_indent = current_indent

                if current_indent > previous_indent and current_paragraph:
                    result.append({
                        "document_name": document_name,
                        "page_number": page_number + 1,
                        "paragraph_number": paragraph_number,
                        "text": " ".join(current_paragraph)
                    })
                    paragraph_number += 1
                    current_paragraph = []

                if text:
                    current_paragraph.append(text)

                previous_indent = current_indent

            if current_paragraph:
                result.append({
                    "document_name": document_name,
                    "page_number": page_number + 1,
                    "paragraph_number": paragraph_number,
                    "text": " ".join(current_paragraph)
                })

    with open('../jsons/elasticsearch_data.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)


In [7]:
parse_pdf_to_paragraphs(file_path)